### Libraries

In [105]:
########## libraries ###########
import pandas as pd
import os
import pickle
import datarobot as dr
import caixautils as cu
import sys,os,os.path
from datarobot.models.modeljob import wait_for_async_model_creation
import numpy as np
from sklearn.metrics import roc_auc_score
sys.path.insert(0, f'..{os.sep}..{os.sep}resources{os.sep}libraries')


In [106]:
# connection a la kde 
esquema = 'xxxxxx'
# CONNECTION
connection = cu.get_datapool_connection(username = 'xxxxxxxxx', password = 'xxxxxx')

In [107]:
# conection dataset
df_20200219 =cu.get_datapool_table(connection, 'xxxxxxx', table_name= 'tx_dataset')

In [113]:
for i in df_20200219.columns:
    print(i)

tot_oper_1dia
tot_oper_3dia
tot_oper_7dia
tot_oper_mes
mediadiaoper_3dia
mediadiaoper_7dia
mediadiaoper_mes
totoperecom_1dia
totoperecom_3dia
totoperecom_7dia
totoperecom_mes
totoperreinte_1dia
totoperreinte_3dia
totoperreinte_7dia
totoperreinte_mes
totfallbanda_1dia
totfallbanda_3dia
totfallbanda_7dia
totfallbanda_mes
totfallmanual_1dia
totfallmanual_3dia
totfallmanual_7dia
totfallmanual_mes
totcontcless_1dia
totcontcless_3dia
totcontcless_7dia
totcontcless_mes
totmanual_1dia
totmanual_3dia
totmanual_7dia
totmanual_mes
totbanda_1dia
totbanda_3dia
totbanda_7dia
totbanda_mes
totentmanual_1dia
totentmanual_3dia
totentmanual_7dia
totentmanual_mes
ratecom_1dia
ratecom_3dia
ratecom_7dia
ratecom_mes
ratreinte_1dia
ratreinte_3dia
ratreinte_7dia
ratreinte_mes
ratfallbanda_1dia
ratfallbanda_3dia
ratfallbanda_7dia
ratfallbanda_mes
ratfallmanual_3dia
ratfallmanual_7dia
ratfallmanual_mes
ratcontcless_1dia
ratcontcless_3dia
ratcontcless_7dia
ratcontcless_mes
ratmanual_1dia
ratmanual_3dia
ratmanual_

In [109]:
columns_drop=['bin',
'num_contrato',
'num_plastico',
'pk_pan',
'pk_bin',
'pk_plastico',
'pk_contrato' 
        
 # ,fk_numperso            
             ]

In [98]:
df_20200219.drop('fecha_operacion',axis='columns' ,inplace=True)

In [112]:
# df_20200219.drop(columns_drop,axis='columns' ,inplace=True)

## DataRobot

In [91]:
TOKEN = 'xxxxxxxxxxxx'
ENDPOINT="xxxxxxxxxxxx"
os.environ['http_proxy']="http://proxyserv.xxxxxxx"
os.environ['https_proxy']="https://proxyserv.xxxxxxxxx"
#export HTTP_PROXY="https://proxyserv.svb.xxxxxxxx"

In [114]:
## Parameters
N_ROWS = 1000000
SAMPLE_PCT=100
RECOMMENDATION_TYPE = dr.enums.RECOMMENDED_MODEL_TYPE.FAST_ACCURATE

def connect_to_dr(token, endpoint):
    '''
    connect to datarobot with you user
    '''
    dr.Client(token=token, endpoint=endpoint)
    

def create_project(df,project_name):
    '''
    df: dataframe/dataset to upload
    '''
    try:
        project = dr.Project.create(df, project_name=project_name)
        return project
    except ValueError:
        pass
    
    
def run_project_dr(target,project):
    '''
    Launch the DataRobot project in Autopilot mode
    '''
    project.set_target(target=str(target),
                       worker_count = -1)

    return project.wait_for_autopilot()
    
    

def unlock_holdout(project):
    project.unlock_holdout()


def select_model_dr(project,recommendation_type=RECOMMENDATION_TYPE):
    '''
    retrieve a specific model
    '''
    model = dr.ModelRecommendation.get(project.id, recommendation_type).get_model()
    return model
    

def retrain_model(project, model, sample_pct=SAMPLE_PCT, frozen=True):
    '''
    With the "frozen" option, all model hyperparameters are frozen.
    '''
    try:
        model_job=model.request_frozen_model(sample_pct=sample_pct)
        #get frozen model id
        if frozen is True:
            frozen_model = wait_for_async_model_creation(
                project_id=model.project_id,
                model_job_id=model_job.id,)
            #save frozen model id
        return frozen_model.id
    except:
        print('Can t retrain the model with {}% of the data'.format(sample_pct))
        
        
        
        
def predictions_model(project, target, model, dataframe):  
    '''
    path_dataset_val: features validation
    target_ val : file target validation
    '''
    # create dataframe
    predictions=pd.DataFrame()
    df_target=pd.DataFrame()
    
    #file_target='target_{}'.format(target) # name outfile target
    #file_predictions='prediction_{}'.format(target) # name outfile prediction
    # read dataset validation
   # df_reader = pd.read_csv(path_dataset_val, compression="gzip", chunksize=n_rows)
   
    
  ##   for j, val_features_data in enumerate(df_reader):
  #  val_features_data.columns = map(str.upper, val_features_data.columns)
  #  val_features_data.set_index('PK_NUMPERSO', inplace=True)

    # val_target=preprocess_target(target_val)
   # val_data_joined = val_features_data.join(val_target, how='left')
    ## fillna
    #null_values(val_data_joined)
   # df_target=df_target.append(val_data_joined[[target]], ignore_index=True)
    #print(val_data_joined.shape)
  #  df_target.to_csv('{}.csv'.format(file_target), sep=';', index=False)  
   # print("{} - Uploading dataset {}".format(datetime.now(), target))
  #  preprocess_val_data_joined =preprocess_features(val_data_joined)

    uploaded_test_dataset = project.upload_dataset(dataframe) # function to preprocess features
    #print("{} - Computing predictions".format(datetime.now()))
    predict_job = model.request_predictions(uploaded_test_dataset.id)
    predict_job.wait_for_completion()
    predictions_df = predict_job.get_result_when_complete(max_wait=1000)
    predictions=predictions.append(predictions_df[['positive_probability']], ignore_index=True)
   # predictions.to_csv('{}.csv'.format(file_predictions), sep=';', index=False) 
      #  break
    #df_target_predictions=pd.DataFrame()
    df_target_predictions=dataframe.copy()
    df_target_predictions['positive_probability']=predictions  
 #   df_target_predictions.fillna(value=0, inplace=True)
   # df_target_predictions.to_csv('Target_prediction_{}.csv'.format(target), sep=';', index=False)      
    return df_target_predictions
    
    

**Get project by project id**

In [117]:
project1 = dr.Project.get(project_id='xxxxxxxxxxxxxxxxxxxxxxxxxx')

**Retrieve a specific model**  

In [128]:
model_id='5e5697acb2d0a13b07ac1221'
model1=dr.Model.get(project1, model_id)

/shared/apps/dtpjpt/dshubnfs/workspace/u0197494/.conda/envs/ntbmkt-ml360_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataRobotDeprecationWarning: `Using a project instance in model.get` has been deprecated in `v2.3`, will be removed in `v3.0`. Please use a project ID instead
  


In [ ]:
predictions_model(project1, df_20200219.ind_reclamada, model1, df_20200219)

In [20]:
connect_to_dr(token=TOKEN, endpoint=ENDPOINT)

In [101]:
project=create_project(df_20200219, '20200219-Payments')

In [68]:
project_2=create_project(df_20200219_v1, '20200219_v1-Payments')

In [32]:
run_project_dr('ind_reclamada',project)

In progress: 8, queued: 18 (waited: 0s)
In progress: 8, queued: 18 (waited: 1s)
In progress: 8, queued: 18 (waited: 2s)
In progress: 8, queued: 18 (waited: 3s)
In progress: 8, queued: 18 (waited: 4s)
In progress: 8, queued: 18 (waited: 6s)
In progress: 8, queued: 18 (waited: 10s)
In progress: 8, queued: 17 (waited: 17s)
In progress: 8, queued: 17 (waited: 31s)
In progress: 7, queued: 17 (waited: 51s)
In progress: 8, queued: 15 (waited: 72s)
In progress: 8, queued: 15 (waited: 93s)
In progress: 8, queued: 14 (waited: 113s)
In progress: 8, queued: 14 (waited: 134s)
In progress: 8, queued: 14 (waited: 155s)
In progress: 8, queued: 13 (waited: 175s)
In progress: 8, queued: 12 (waited: 196s)
In progress: 7, queued: 12 (waited: 216s)
In progress: 8, queued: 11 (waited: 237s)
In progress: 8, queued: 10 (waited: 258s)
In progress: 8, queued: 10 (waited: 278s)
In progress: 8, queued: 9 (waited: 299s)
In progress: 8, queued: 9 (waited: 319s)
In progress: 8, queued: 9 (waited: 340s)
In progress: 

In [65]:
run_project_dr('ind_reclamada',project2)

NameError: name 'project2' is not defined

In [64]:
df_20200219_v1.ind_reclamada

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
865337    0
865338    0
865339    0
865340    0
865341    0
865342    0
865343    0
865344    0
865345    0
865346    0
865347    0
865348    0
865349    0
865350    0
865351    0
865352    0
865353    0
865354    0
865355    0
865356    0
865357    0
865358    0
865359    0
865360    0
865361    0
865362    0
865363    0
865364    0
865365    0
865366    0
Name: ind_reclamada, Length: 865367, dtype: int64

## Prdicciones en DR

In [103]:
len(df_20200219.columns)

115

In [1]:

y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])


In [2]:
roc_auc_score(y_true, y_scores)

0.75